In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

%run ../../deepfaune/deepfaune_model.py

In [ ]:
weights_path = '../../deepfaune/models/deepfaune-vit_large_patch14_dinov2.lvd142m.v4.pt'
model = Deepfaune(weights_path)

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q", "v"],
    lora_dropout=0.05, bias="none", task_type=TaskType.CAUSAL_LM
)

# Wrap the ViT model
model = get_peft_model(model.base_model, lora_config)

# Only LoRA parameters are trainable
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Training loop same as above
for imgs, labels in dataloader:
    imgs, labels = imgs.to(model.device), labels.to(model.device)
    optimizer.zero_grad()
    outputs = model(imgs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()